In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
%matplotlib inline

from datetime import datetime, time
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sknn.mlp import Classifier, Layer
from sklearn.linear_model import LinearRegression, RidgeClassifierCV, LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import *
from sklearn.cross_validation import KFold, train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import log_loss
from sklearn.grid_search import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn import cross_validation

In [2]:
# def get_data():
df = pd.read_csv('train.csv.gz')

In [3]:
# df.total_time.map(lambda x: np.log10(x+1)).hist()

In [4]:
df_color = pd.DataFrame({'color1':df.Color.str.split('/').str[0], 'color2':df.Color.str.split('/').str[1]})
vc_color = pd.concat((df_color['color1'], df_color['color2'])).value_counts()
common_colors = vc_color[vc_color > 300].index

In [87]:
def get_vars(df, cols_x, cols_y=None):
    minmax = MinMaxScaler()
    
    df.drop_duplicates(inplace=True)
    df.DateTime = pd.to_datetime(df.DateTime)
    df['weekday'] = df.DateTime.dt.weekday
    df['weekend'] = (df.weekday > 4) * 1
    df['viralata'] = (df.Breed.str.contains('Mix') | df.Breed.str.contains('/')) *1
    df['named'] = ~df.Name.isnull() * 1
    df['common_colors'] = df.Color.isin(common_colors) * 1
    df['sex'] = df.SexuponOutcome.str.split(' ').str[1]
    df['castrado'] = (df.SexuponOutcome.str.split(' ').str[0].str.contains('Neutered') | df.SexuponOutcome.str.split(' ').str[0].str.contains('Spayed')) * 1
    df['first_color'] = df.Color.str.split('/').str[0].str.split(' ').str[0]
    df['raca'] = df.Breed.str.replace('Mix','').str.split('/').str[0].str.strip()
    #     df.ix[df.AgeuponOutcome.isnull(),'AgeuponOutcome'] = 0
    filt = df.AgeuponOutcome.str.contains('year')
    df['days_multiplyer'] = 0
    df.ix[df.AgeuponOutcome.str.contains('year')==True,'days_multiplyer'] = 365
    df.ix[df.AgeuponOutcome.str.contains('month')==True,'days_multiplyer'] = 30
    df.ix[df.AgeuponOutcome.str.contains('week')==True,'days_multiplyer'] = 7
    df.ix[df.AgeuponOutcome.str.contains('day')==True,'days_multiplyer'] = 1
    df.totaltime = df.AgeuponOutcome.str.split().str[0]
    df.ix[df.totaltime.isnull(),'AgeuponOutcome'] = 0
    df['total_time'] = df.totaltime.astype('float64') * df.days_multiplyer
    
    df.ix[df.total_time.isnull(), 'total_time'] = 0
#     df.total_time = df.total_time.map(lambda x: np.log10(x+1))
    df['mixed_color'] = df.Color.str.contains('/').astype('int')
    
    df['year'] = df.DateTime.dt.year
    df['month'] = df.DateTime.dt.month
    df['yearmonth'] = df.DateTime.dt.month + df.DateTime.dt.year
    df['day'] = df.DateTime.dt.day
    df['time'] = df.DateTime.dt.hour * 60 + df.DateTime.dt.minute
#     df.time = df.time.map(lambda x: np.log10(x+1))
    
    df['name_len'] = df.Name.str.len()
    df.ix[df.name_len.isnull(), 'name_len'] = df.name_len.median()
    
    X = pd.get_dummies(df[cols_x]).values
    if cols_y:            
        df.ix[df.OutcomeType == 'Return_to_owner', 'IndY'] = 1
        df.ix[df.OutcomeType == 'Euthanasia', 'IndY'] = 2
        df.ix[df.OutcomeType == 'Adoption', 'IndY'] = 3
        df.ix[df.OutcomeType == 'Transfer', 'IndY'] = 4
        df.ix[df.OutcomeType == 'Died', 'IndY'] = 5
        df.sort_values(by='OutcomeType')        
        return X, df[cols_y].values
    else:
        return df
    

In [6]:
# df.index = [df.index,df.Color]

In [7]:
# df.ix[df.mixed_color.isin(df.mixed_color.value_counts()[df.mixed_color.value_counts() == True].index)]['OutcomeType'].value_counts().plot(kind='bar')

In [8]:
cols_x = [
    'AnimalType','SexuponOutcome','viralata',
    'total_time','weekday','weekend',
    'common_colors','mixed_color',
    'year', 'name_len', 'month', 
    'day', 'time','named',
    'sex','castrado','first_color','raca']
cols_y = ['OutcomeType']

In [9]:
cols_1 = [
    'AnimalType',
    'viralata','weekend','total_time','time','name_len','year','month',
    'named','sex','castrado','first_color','raca'
]
# cols_1 = ['common_colors','SexuponOutcome','mixed_color','AnimalType','viralata','weekend','year']
cols_rf = ['total_time', 'SexuponOutcome', 'AnimalType']
cols_extra = ['total_time','time', 'SexuponOutcome', 'AnimalType']
cols_gb = ['total_time','time','name_len', 'month', 'AnimalType', 'SexuponOutcome']

In [10]:
X, Y = get_vars(df, cols_x, cols_y)
X = pd.get_dummies(df[cols_1])
gb_features = X.columns
Y = Y.reshape(Y.shape[0])

In [11]:
# df.ix[df.total_time.isnull()]

In [12]:
df.shape[0]/3

8909.666666666666

In [13]:
kf = StratifiedKFold(df.OutcomeType,n_folds=2, shuffle=True, random_state=500)
# a = np.concatenate((list(kf)[0][0], list(kf)[1][0]))#, list(kf)[2][0], list(kf)[3][0]))
a = list(kf)[0][0]#, list(kf)[2][0], list(kf)[3][0]))
b = list(kf)[1][0]
# c = list(list(kf)[2][0])[0:8909]
# d = list(list(kf)[3][0])[0:6682]


# X_knn = X[train]
# Y_knn = Y[train]

# X_gb = X[train]
# Y_gb = Y[train]

# X_forest = X[train]
# Y_forest = Y[train]

# X_test = X[test]
# Y_test = Y[test]

# df_knn = df.ix[train]
# df_gb = df.ix[train]
# df_forest = df.ix[train]

# df_train = df

In [14]:
# train_x = X_train
# train_y = Y_train
# test_x = X_test
# test_y = Y_test

In [15]:
def train_regression(forest, train_x, train_y, test_x=None, test_y=None):
    nfolds = 5
    ini = datetime.now()

    kf = StratifiedKFold(train_y, n_folds=nfolds, shuffle=True, random_state=500)
    log_acc = 0
    print('Training linear...')
    for k, (train, test) in enumerate(kf, start=1):
        forest.fit(train_x[train], train_y[train])
        pred = forest.predict(train_x[test]) 
        del pred
        
#     log_acc = 0
#     print('Training ridge...')
#     forest.fit(train_x, train_y)
#     pred = forest.predict(train_x) 
#     del pred
    
    fim = datetime.now()
    print(fim - ini)
    return forest

In [16]:
def train_classifier(forest, train_x, train_y, nfolds, test_x=None, test_y=None):
    ini = datetime.now()

    kf = StratifiedKFold(train_y, n_folds=nfolds, shuffle=True)
#     log_acc = 0
    print('Training classifier...')
    for k, (train, test) in enumerate(kf, start=1):
        forest.fit(train_x[train], train_y[train])
        print(k, ' ', end="")
#         pred = forest.predict_proba(train_x[test])
        
#         loglo = log_loss(train_y[test], pred)
#         log_acc += loglo
#         print('fold:', k, 'log loss:', loglo)
#         del pred
        
        
    scores = cross_validation.cross_val_score(forest, train_x, train_y, cv=5, scoring='log_loss')
    print("Log loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    
#     print('acc:',log_acc/nfolds)
#     forest.fit(train_x, train_y)
#     print('final log loss: ',log_loss(train_y, forest.predict_proba(train_x)))
#     print('\n\n')
#     log_acc = 0
#     print('Training rf...')
#     forest.fit(train_x, train_y)
#     pred = forest.predict_proba(train_x) 
#     loglo = log_loss(train_y, pred)
#     log_acc += loglo
#     print('log loss:', loglo)
#     del pred

    fim = datetime.now()
    print(fim - ini)
    print('')
    return forest

In [17]:
# minmax1 = MinMaxScaler()
# minmax1 = minmax1.fit(pd.get_dummies(df[cols_1]).values)

In [18]:
# minmax4 = MinMaxScaler()
# minmax4 = minmax4.fit(pd.get_dummies(df[cols_4]).values)

In [137]:
def train(x, y, y_ind):    
    x1 = x
    #y=y[a]
#     x4 = pd.get_dummies(x[cols_4]).values
#     x1_minmax = minmax1.transform(x1)
#     x4_minmax = minmax4.transform(x4)
    x1_log2 = np.log2(x1+1)
    #x1_log10 = np.log10(x1+1)
    
#     xrf = pd.get_dummies(x[cols_rf]).values
#     xrf = np.log2(xrf+1)
#     xextra = pd.get_dummies(x[cols_extra]).values
#     xextra = np.log10(xextra+1)
#     xgb = pd.get_dummies(x[cols_gb]).values
        
    if type(rf_features) != type(None):  
        xrf = pd.get_dummies(x)[rf_features].values[a]
        print('new rf')
    else:
        xrf = x1
    xrf = np.log2(xrf+1)
    
    if type(extra_features) != type(None):        
        xextra = pd.get_dummies(x)[extra_features].values[a]
        print('new extra')
    else:
        xextra = x1
    xextra = np.log10(xextra+1)
    
    if type(gb_features) != type(None):
        xgb = pd.get_dummies(x)[gb_features].values
        print('new gb')
    else:
        xgb = x1
    
    
    nfolds = 2
#     kf = StratifiedKFold(y, n_folds=nfolds, shuffle=True)
    
    extra = ExtraTreesClassifier(n_estimators=400, max_depth=50,max_leaf_nodes=60, n_jobs=4, random_state=500)    
    
    rf_gini = RandomForestClassifier(
        n_estimators=400,
        max_depth=75,
        max_leaf_nodes=40,
        n_jobs=4,
        random_state=500,
        criterion='gini')
        
    x1_log10 = np.log10(x.values[a]+1)
    knn = KNeighborsClassifier(500)
    gb = GradientBoostingClassifier(min_samples_split=1000, max_depth=6, random_state=500)
    
#     for k, (train, test) in enumerate(kf, start=1):
        
    #extra = train_classifier(extra, xextra, y, 5)
#         ens = gb1.predict_proba(x1_minmax[train])
#         ens = np.concatenate((x1_minmax[train], ens), axis=1)

    #rf_gini = train_classifier(rf_gini, xrf, y, 10)
#     rf_entropy = train_classifier(rf_entropy, x1_log2, y, 10)
    
#         ens = gb1.predict_proba(x1_minmax[train])
#         ens = np.concatenate((x1_minmax[train], ens), axis=1)

    #knn = train_classifier(knn, x1_log10, y, 4)
#         ens = gb1.predict_proba(x1_minmax[train])
#         ens = np.concatenate((x1_minmax[train], ens), axis=1)

#         ridge = RidgeClassifierCV()
#         ridge = train_regression(ridge, x1_minmax[train], y[train])
#         ens = ens.reshape(ens.shape[0],1)

#         ens = np.concatenate((x1_minmax[train], ens), axis=1)
    gb = train_classifier(gb, xgb, y, 5)
    print('\n')
#     pass
    
            

#     gb = train_gb(np.concatenate((ens1, ens2, x4),axis=1), y)

    #         proba = gb.predict_proba(np.concatenate((ens, X[train]),axis=1))
        
#     return extra, rf_gini, knn, gb


    return extra, rf_gini, knn, gb

In [138]:
def pred(extra, rf, knn, gb, x):
    x1 = x.values[a]
#     x4 = pd.get_dummies(x[cols_4]).values
    
#     x1_minmax = minmax1.transform(x1)
#     x4_minmax = minmax4.transform(x4)
#     x1_log2 = np.log2(x1+1)
    x1_log10 = np.log10(x1+1)   
    
    if type(rf_features) != type(None):  
        xrf = pd.get_dummies(x)[rf_features].values[a]
        print('new rf')
    else:
        xrf = x1
#     xrf = np.log2(xrf+1)
    
    if type(extra_features) != type(None):        
        xextra = pd.get_dummies(x)[extra_features].values[a]
        print('new extra')
    else:
        xextra = x1
#     xextra = np.log10(xextra+1)
    
    if type(gb_features) != type(None):
        xgb = pd.get_dummies(x)[gb_features].values[a]
        print('new gb')
    else:
        xgb = x1
    
    
    one = extra.predict_proba(xextra)
    two = rf.predict_proba(xrf)
    three = knn.predict_proba(x1_log10)
#     ens = ens.reshape(ens.shape[0],1)
#     ens = np.concatenate((x4, ens), axis=1)
    four = gb.predict_proba(xgb)
#     return gb.predict_proba(ens)
    newx = [one, two, three, four]
    return newx
#     return gb.predict_proba(np.concatenate((ens1,ens2, x4),axis=1))

In [139]:
# pd.DataFrame(extra.feature_importances_).plot(kind='bar')

In [140]:
#extra_features = None
#rf_features = None
#gb_features = None

In [141]:
extra, rf, knn, gb = train(
    X,
    Y,
    df.IndY.loc[a].values
)


new rf
new extra
new gb
Training classifier...


KeyboardInterrupt: 

In [ ]:
# 0.98
# 0.97
# 0.94
# 0.77

# 0.75

In [ ]:
# del extra_features
# del rf_features
# del gb_features
# extra_features = None
# rf_features = None
# gb_features = None
# features = 200

In [ ]:
# features -= 1
rfe = RFE(extra)
rfe.fit(X[extra_features].values[a], Y[a])
extraf = pd.DataFrame(rfe.support_, columns=['vars'])
extraf.index = pd.get_dummies(df[cols_1]).columns
extraf.plot(kind='bar')
extra_features = extraf.ix[extraf['vars'] == True, 'vars'].index

In [ ]:
rfe = RFE(rf)
rfe.fit(X.values[a], Y[a])
rff = pd.DataFrame(rfe.support_, columns=['vars'])
rff.index = pd.get_dummies(df[cols_1]).columns
rff.plot(kind='bar')
rf_features = rff.ix[rff['vars'] == True, 'vars'].index

In [154]:
rfe = RFE(gb)
rfe.fit(X[gb_features].values, Y)
gbf = pd.DataFrame(rfe.support_, columns=['vars'])
gbf.index = pd.get_dummies(df[cols_1]).columns
gbf.plot(kind='bar')
gb_features = gbf.ix[gbf['vars'] == True, 'vars'].index

ValueError: Length mismatch: Expected axis has 131 elements, new values have 262 elements

In [145]:
# last_valid_features = gb_features

In [ ]:
# seiya = pd.DataFrame(gb.feature_importances_)
# seiya.plot(kind='bar')

In [ ]:
# res = pred(
#     extra,
#     rf,
#     knn,
#     gb, 
#     df.loc[b]
#           )
# prd = clf.predict_proba(res)

In [ ]:
losses = [.3, .3, 0, .4]

In [ ]:
voting = VotingClassifier([
        ('extra', extra), 
        ('rf', rf), 
#         ('knn', knn), 
        ('gb', gb)
    ],
    weights=[
        1,
        1,
#         0,
        1.2
    ],
    voting='soft')

In [ ]:
voting.fit(
    pd.get_dummies(df[cols_1].loc[a]).values,
    Y[a]
)
scores = cross_validation.cross_val_score(voting, pd.get_dummies(df[cols_1].loc[b]).values,Y[b], cv=5, scoring='log_loss')
print("Log loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
# prd = voting.predict_proba(pd.get_dummies(df[cols_1].loc[b]).values)

In [ ]:
res = pred(
    extra,
    rf,
    knn,
    gb, 
    df.loc[b]
          )

In [ ]:
prd = (res[0] * losses[0]) + (res[1] * losses[1]) + (res[2] * losses[2]) + (res[3] * losses[3])

In [ ]:
log_loss(Y[b], prd)

In [ ]:
# 0.7651903888988153

In [88]:
def get_test():
    df_test = pd.read_csv('test.csv.gz')    
    X_test = get_vars(df_test, cols_x)

    return X_test

In [89]:
X_test = get_test()
X_test = pd.get_dummies(X_test)

In [125]:
missing_cols = list(gb_features - X_test.columns)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


In [127]:
for col in missing_cols:
    X_test[col] = 0

In [129]:
cats = df.OutcomeType.unique()
cats.sort()
final_result = gb.predict_proba(X_test[gb_features].values)
final_result = pd.DataFrame(final_result, columns=cats)

In [ ]:
cats = df.OutcomeType.unique()
cats.sort()

first, second, thirdth, fourth  = pred(extra, rf, knn, gb, X_test)
final_result = first*one + second*two +  fourth*four # +thirdth*three

# final_result = pred(extra, gb, X_test)
final_result = pd.DataFrame(final_result, columns=cats)

In [131]:
final_result['Id'] = final_result.index+1
final_result.set_index('Id', inplace=True)

In [132]:
today = datetime.today()
t = today.strftime('%Y%m%d%H%M')
final_result.to_csv('result_%s.csv.gz'%t, compression='gzip')